### Notebook Description

---
- This notebook details the procedures of feature selection using RFECV and BORUTA.
- The features selected through these methods will be utilized in the main analysis notebook for further modeling.
- Recursive Feature Elimination with Cross-Validation (RFECV) was conducted twice, each time with the same sample size of 30,000 but using different evaluation metrics. These metrics were chosen to emphasize recall for class one, while still maintaining a balanced output and the ability to effectively distinguish between the two classes.
- From an initial set of 150 features, BORUTA recommended retaining 71 features, whereas RFECV suggested only 4.
- The potential use of feature weights is being considered, particularly to increase the weight of features identified by RFECV.
- Given the extensive nature of the feature selection process, the results have been prominently displayed under distinct headers, with the detailed procedures minimized to reduce verbose output.

---

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from boruta import BorutaPy
from lazypredict.Supervised import LazyClassifier
from sklearn.decomposition import PCA
from joblib import parallel_backend

In [2]:
data_features_selection = pd.read_csv('data_features_selection.csv')

### RFECV 1 - 30,000 sample, F1 Score, F2 Score and AUC Combination. 

- Recursive Feature Elimination with Cross-Validation (RFECV), using custom scorer using f1 score, f2 score, and ROC-AUC. 

In [48]:
X = data_features_selection.drop('TARGET', axis=1)
y = data_features_selection['TARGET']

X_sample, _, y_sample, _ = train_test_split(
    X, y, train_size=30000, random_state=32, stratify=y)


def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    beta = 2
    f2 = (1 + beta**2) * (precision * recall) / \
        ((beta**2 * precision) + recall)
    roc_auc = roc_auc_score(y, estimator.predict_proba(X)[:, 1])
    return 0.2 * f1 + 0.4 * f2 + 0.4 * roc_auc


rf_classifier = RandomForestClassifier(n_jobs=-1, verbose=1)

rfecv = RFECV(
    estimator=rf_classifier,
    step=1,
    cv=StratifiedKFold(5),
    scoring=custom_scorer,
    verbose=1,
    n_jobs=-1
)

rfecv.fit(X_sample, y_sample)
selected_features = X_sample.columns[rfecv.support_]
print("Selected features:", selected_features)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.0s finished
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
/home/user/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classifi

Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting estimator with 152 features.
Fitting estimator with 151 features.
Fitting estimator with 150 features.
Fitting estimator with 149 features.
Fitting estimator with 148 features.
Fitting estimator with 147 features.
Fitting estimator with 146 features.
Fitting estimator with 145 features.
Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
F

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]

Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting estimator with 152 features.
Fitting estimator with 151 features.
Fitting estimator with 150 features.
Fitting estimator with 149 features.
Fitting estimator with 148 features.
Fitting estimator with 147 features.
Fitting estimator with 146 features.
Fitting estimator with 145 features.
Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
F

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


Fitting estimator with 5 features.


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.1s


Selected features: Index(['DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2'], dtype='object')


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.6s finished


### Results

---
- Selected features: 'DAYS_BIRTH', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_2'
- These are ones that contributed most to the specified scorer. 
- There is possibility of using also other scorers to evaluate features. 

---

### RFECV 2 - 30,000 sample, F2 score and AUC combination 

In [3]:
X = data_features_selection.drop('TARGET', axis=1)
y = data_features_selection['TARGET']

X_sample, _, y_sample, _ = train_test_split(
    X, y, train_size=30000, random_state=32, stratify=y)

def custom_scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    recall = recall_score(y, y_pred)
    precision = precision_score(y, y_pred)
    beta = 3  
    f2 = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    roc_auc = roc_auc_score(y, estimator.predict_proba(X)[:, 1])
    return 0.3 * f2 + 0.7 * roc_auc


rf_classifier = RandomForestClassifier(n_jobs=-1, verbose=1)

rfecv = RFECV(
    estimator=rf_classifier,
    step=1,
    cv=StratifiedKFold(5),
    scoring=custom_scorer,
    verbose=1,
    n_jobs=-1
)

rfecv.fit(X_sample, y_sample)
selected_features = X_sample.columns[rfecv.support_]
print("Selected features:", selected_features)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    9.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Para

Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting estimator with 152 features.
Fitting estimator with 151 features.
Fitting estimator with 150 features.
Fitting estimator with 149 features.
Fitting estimator with 148 features.
Fitting estimator with 147 features.
Fitting estimator with 146 features.
Fitting estimator with 145 features.
Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
F

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]

Fitting estimator with 154 features.
Fitting estimator with 153 features.
Fitting estimator with 152 features.
Fitting estimator with 151 features.
Fitting estimator with 150 features.
Fitting estimator with 149 features.
Fitting estimator with 148 features.
Fitting estimator with 147 features.
Fitting estimator with 146 features.
Fitting estimator with 145 features.
Fitting estimator with 144 features.
Fitting estimator with 143 features.
Fitting estimator with 142 features.
Fitting estimator with 141 features.
Fitting estimator with 140 features.
Fitting estimator with 139 features.
Fitting estimator with 138 features.
Fitting estimator with 137 features.
Fitting estimator with 136 features.
Fitting estimator with 135 features.
Fitting estimator with 134 features.
Fitting estimator with 133 features.
Fitting estimator with 132 features.
Fitting estimator with 131 features.
Fitting estimator with 130 features.
Fitting estimator with 129 features.
Fitting estimator with 128 features.
F

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[P

...

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


Fitting estimator with 32 features.


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.3s


Selected features: Index(['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
       'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'LIVINGAREA_AVG',
       'YEARS_BEGINEXPLUATATION_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'LANDAREA_MEDI',
       'LIVINGAREA_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object')


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.1s finished


### Results

---
- Selected features shown bellow.
- These are ones that contributed most to the specified scorer of 0.3 * F2 score with beta=3, and 0.7 * ROC-AUC. 
- This time we got more features, having now outputs of these two RFECV's we can add additional weights or somehow prioritize thse features with hyperparameters in the models that get built.
- All of the features from first RFECV are present in this one as well.
---

In [4]:
features_from_second_refcv = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE',
       'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'LIVINGAREA_AVG',
       'YEARS_BEGINEXPLUATATION_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'LANDAREA_MEDI',
       'LIVINGAREA_MEDI', 'TOTALAREA_MODE', 'OBS_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR']

In [5]:
len(features_from_second_refcv)

31

In [9]:
#features_df = pd.DataFrame(features_from_second_refcv, columns=['Feature'])
#features_df.to_csv('features_list_RFECV_2.csv', index=False)

### BORUTA sample test

In [5]:
X_sample, _, y_sample, _ = train_test_split(
    data_features_selection.drop('TARGET', axis=1),
    data_features_selection['TARGET'],
    train_size=5000,
    random_state=32,
    stratify=data_features_selection['TARGET']
)

X_sample = X_sample.values
y_sample = y_sample.values.ravel()

rf = RandomForestClassifier(
    n_jobs=-1, class_weight='balanced', max_depth=5, verbose=500)
boruta_selector = BorutaPy(rf, n_estimators='auto', random_state=1, verbose=50)

boruta_selector.fit(X_sample, y_sample)

selected_features = boruta_selector.support_
feature_ranking = boruta_selector.ranking_

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
building tree 1 of 350
building tree 2 of 350
building tree 3 of 350
building tree 4 of 350
building tree 5 of 350
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
building tree 6 of 350
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
building tree 7 of 350
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s
building tree 8 of 350
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
building tree 9 of 350
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
building tree 10 of 350
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
building tree 11 of 350
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.0s
building tree 12 of 350
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
building tree 13 of 350
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
building tree 14 of 350
building tree 15 of 350
[Parallel(

### Results

In [12]:
selected_features = boruta_selector.support_
feature_names = data_features_selection.drop('TARGET', axis=1).columns
selected_feature_names = feature_names[selected_features]
print("Selected Features:")
print(selected_feature_names)

Selected Features:
Index(['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE'],
      dtype='object')


In [10]:
feature_ranking = boruta_selector.ranking_
feature_names = data_features_selection.drop('TARGET', axis=1).columns
ranked_features = list(zip(feature_names, feature_ranking))
ranked_features_sorted = sorted(ranked_features, key=lambda x: x[1])
for feature, rank in ranked_features_sorted:
    print(f"Feature: {feature}, Rank: {rank}")

Feature: DAYS_BIRTH, Rank: 1
Feature: DAYS_EMPLOYED, Rank: 1
Feature: DAYS_ID_PUBLISH, Rank: 1
Feature: EXT_SOURCE_1, Rank: 1
Feature: EXT_SOURCE_2, Rank: 1
Feature: EXT_SOURCE_3, Rank: 1
Feature: DAYS_LAST_PHONE_CHANGE, Rank: 1
Feature: DAYS_REGISTRATION, Rank: 2
Feature: TOTALAREA_MODE, Rank: 2
Feature: OWN_CAR_AGE, Rank: 3
Feature: AMT_GOODS_PRICE, Rank: 4
Feature: AMT_CREDIT, Rank: 5
Feature: REGION_POPULATION_RELATIVE, Rank: 6
Feature: LIVINGAREA_MEDI, Rank: 7
Feature: REGION_RATING_CLIENT_W_CITY, Rank: 8
Feature: LIVINGAREA_AVG, Rank: 9
Feature: AMT_INCOME_TOTAL, Rank: 11
Feature: APARTMENTS_MEDI, Rank: 11
Feature: APARTMENTS_AVG, Rank: 13
Feature: LIVINGAREA_MODE, Rank: 13
Feature: AMT_ANNUITY, Rank: 15
Feature: LANDAREA_AVG, Rank: 15
Feature: LANDAREA_MEDI, Rank: 16
Feature: REGION_RATING_CLIENT, Rank: 18
Feature: COMMONAREA_MODE, Rank: 18
Feature: COMMONAREA_AVG, Rank: 19
Feature: COMMONAREA_MEDI, Rank: 20
Feature: BASEMENTAREA_AVG, Rank: 21
Feature: BASEMENTAREA_MEDI, Rank: 2

### BORUTA All data

In [15]:
X = data_features_selection.drop('TARGET', axis=1).values
y = data_features_selection['TARGET'].values.ravel()
rf = RandomForestClassifier(
    n_jobs=-1, class_weight='balanced', max_depth=5, verbose=2)
boruta_selector = BorutaPy(rf, n_estimators='auto', random_state=1, verbose=1)
boruta_selector.fit(X, y)
selected_features = boruta_selector.support_
feature_ranking = boruta_selector.ranking_

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 350building tree 2 of 350

building tree 3 of 350
building tree 4 of 350
building tree 5 of 350
building tree 6 of 350
building tree 7 of 350
building tree 8 of 350
building tree 9 of 350
building tree 10 of 350
building tree 11 of 350
building tree 12 of 350
building tree 13 of 350
building tree 14 of 350
building tree 15 of 350
building tree 16 of 350
building tree 17 of 350
building tree 18 of 350
building tree 19 of 350
building tree 20 of 350
building tree 21 of 350
building tree 22 of 350
building tree 23 of 350
building tree 24 of 350
building tree 25 of 350
building tree 26 of 350
building tree 27 of 350
building tree 28 of 350
building tree 29 of 350
building tree 30 of 350
building tree 31 of 350
building tree 32 of 350
building tree 33 of 350
building tree 34 of 350
building tree 35 of 350
building tree 36 of 350
building tree 37 of 350


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.3s


building tree 38 of 350
building tree 39 of 350
building tree 40 of 350
building tree 41 of 350
building tree 42 of 350
building tree 43 of 350
building tree 44 of 350
building tree 45 of 350
building tree 46 of 350
building tree 47 of 350
building tree 48 of 350
building tree 49 of 350
building tree 50 of 350
building tree 51 of 350
building tree 52 of 350
building tree 53 of 350
building tree 54 of 350
building tree 55 of 350
building tree 56 of 350
building tree 57 of 350
building tree 58 of 350
building tree 59 of 350
building tree 60 of 350
building tree 61 of 350
building tree 62 of 350
building tree 63 of 350
building tree 64 of 350
building tree 65 of 350
building tree 66 of 350
building tree 67 of 350
building tree 68 of 350
building tree 69 of 350
building tree 70 of 350
building tree 71 of 350
building tree 72 of 350
building tree 73 of 350
building tree 74 of 350
building tree 75 of 350
building tree 76 of 350
building tree 77 of 350
building tree 78 of 350
building tree 79

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   30.1s


...

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.4s


building tree 38 of 249
building tree 39 of 249
building tree 40 of 249
building tree 41 of 249
building tree 42 of 249
building tree 43 of 249
building tree 44 of 249
building tree 45 of 249
building tree 46 of 249
building tree 47 of 249
building tree 48 of 249
building tree 49 of 249
building tree 50 of 249
building tree 51 of 249
building tree 52 of 249
building tree 53 of 249
building tree 54 of 249
building tree 55 of 249
building tree 56 of 249
building tree 57 of 249
building tree 58 of 249
building tree 59 of 249
building tree 60 of 249
building tree 61 of 249
building tree 62 of 249
building tree 63 of 249
building tree 64 of 249
building tree 65 of 249
building tree 66 of 249
building tree 67 of 249
building tree 68 of 249
building tree 69 of 249
building tree 70 of 249
building tree 71 of 249
building tree 72 of 249
building tree 73 of 249
building tree 74 of 249
building tree 75 of 249
building tree 76 of 249
building tree 77 of 249
building tree 78 of 249
building tree 79

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   31.7s


building tree 159 of 249
building tree 160 of 249
building tree 161 of 249
building tree 162 of 249
building tree 163 of 249
building tree 164 of 249
building tree 165 of 249
building tree 166 of 249
building tree 167 of 249
building tree 168 of 249
building tree 169 of 249
building tree 170 of 249
building tree 171 of 249
building tree 172 of 249
building tree 173 of 249
building tree 174 of 249
building tree 175 of 249
building tree 176 of 249
building tree 177 of 249
building tree 178 of 249
building tree 179 of 249
building tree 180 of 249
building tree 181 of 249
building tree 182 of 249
building tree 183 of 249
building tree 184 of 249
building tree 185 of 249
building tree 186 of 249
building tree 187 of 249
building tree 188 of 249
building tree 189 of 249
building tree 190 of 249
building tree 191 of 249
building tree 192 of 249
building tree 193 of 249
building tree 194 of 249
building tree 195 of 249
building tree 196 of 249
building tree 197 of 249
building tree 198 of 249


[Parallel(n_jobs=-1)]: Done 249 out of 249 | elapsed:   56.4s finished


### Results

---
- Out of 150 features, we get 71 Confirmed. 
- Rejected 76 features.
- Undecided  3 features. 

---

In [30]:
feature_names = data_features_selection.drop('TARGET', axis=1).columns

selected_features = boruta_selector.support_
selected_feature_names = feature_names[selected_features]

feature_ranking = boruta_selector.ranking_
ranked_features = list(zip(feature_names, feature_ranking))
ranked_features_sorted = sorted(ranked_features, key=lambda x: x[1])

for feature, rank in ranked_features_sorted:
    status = "Selected" if feature in selected_feature_names else "Not Selected"
    print(f"Feature: {feature}, Rank: {rank}, Status: {status}")

Feature: CODE_GENDER, Rank: 1, Status: Selected
Feature: FLAG_OWN_CAR, Rank: 1, Status: Selected
Feature: AMT_INCOME_TOTAL, Rank: 1, Status: Selected
Feature: AMT_CREDIT, Rank: 1, Status: Selected
Feature: AMT_ANNUITY, Rank: 1, Status: Selected
Feature: AMT_GOODS_PRICE, Rank: 1, Status: Selected
Feature: NAME_EDUCATION_TYPE, Rank: 1, Status: Selected
Feature: REGION_POPULATION_RELATIVE, Rank: 1, Status: Selected
Feature: DAYS_BIRTH, Rank: 1, Status: Selected
Feature: DAYS_EMPLOYED, Rank: 1, Status: Selected
Feature: DAYS_REGISTRATION, Rank: 1, Status: Selected
Feature: DAYS_ID_PUBLISH, Rank: 1, Status: Selected
Feature: OWN_CAR_AGE, Rank: 1, Status: Selected
Feature: FLAG_EMP_PHONE, Rank: 1, Status: Selected
Feature: FLAG_WORK_PHONE, Rank: 1, Status: Selected
Feature: REGION_RATING_CLIENT, Rank: 1, Status: Selected
Feature: REGION_RATING_CLIENT_W_CITY, Rank: 1, Status: Selected
Feature: HOUR_APPR_PROCESS_START, Rank: 1, Status: Selected
Feature: REG_CITY_NOT_LIVE_CITY, Rank: 1, Status:

In [40]:
features_data = [{'Feature': feature,
                  'Rank': rank,
                  'Status': "Selected" if feature in selected_feature_names else "Not Selected"}
                 for feature, rank in ranked_features_sorted]


features_df = pd.DataFrame(features_data)

In [44]:
features_df.head()

,Feature,Rank,Status
0,CODE_GENDER,1,Selected
1,FLAG_OWN_CAR,1,Selected
2,AMT_INCOME_TOTAL,1,Selected
3,AMT_CREDIT,1,Selected
4,AMT_ANNUITY,1,Selected


In [32]:
selected_feature_names

Index(['CODE_GENDER', 'FLAG_OWN_CAR', 'AMT_INCOME_TOTAL', 'AMT_CREDIT',
       'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_EDUCATION_TYPE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG',
       'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
       'YEARS_BEGINEXPLUATATION_MODE', 'ELEVATORS_MODE', 'FLOORSMAX_MODE',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAREA_MODE', 'APARTMENTS_MEDI',
       'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLU

In [31]:
feature_names = data_features_selection.drop('TARGET', axis=1).columns
tentative_features = feature_names[boruta_selector.support_weak_].tolist()
print("Tentative (Uncertain) Features:", tentative_features)

Tentative (Uncertain) Features: ['YEARS_BUILD_AVG', 'LIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE']


In [33]:
tentative_features

['YEARS_BUILD_AVG', 'LIVINGAPARTMENTS_AVG', 'YEARS_BUILD_MODE']

In [37]:
filtered_data = data_features_selection[list(
    selected_feature_names) + ['TARGET']]

In [39]:
#filtered_data.to_csv('filtered_data_after_boruta.csv', index=False)

- Will be using this dataset in Main analysis notebook